In [2]:
import cv2
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
from matplotlib import pyplot as plt
import matplotlib.animation as animation
import imageio
import os

video1 = cv2.VideoCapture('source//birds_in_sky.mp4')
#video1 = cv2.VideoCapture('source//hawk.mp4')
output_folder = 'result'

In [3]:
# Функція для створення GIF з набору зображень

def create_gif(images, output_filename, fps=10):
    """
    Creates a GIF from a list of images.
    
    Args:
        images (list): A list of images (NumPy arrays from OpenCV).
        output_filename (str): The path and name for the output GIF file.
        fps (int): Frames per second for the GIF.
    """
    # imageio очікує зображення у форматі RGB, а OpenCV використовує BGR.
    # Тому ми конвертуємо колір для кожного зображення.
    rgb_images = [cv2.cvtColor(img, cv2.COLOR_BGR2RGB) for img in images]
    
    # Створюємо GIF
    imageio.mimsave(output_filename, rgb_images, fps=fps)
    print(f"Successfully saved GIF to {output_filename}")

In [4]:
# ShiTomasi corner detection
config_st = {'maxCorners': 100,
             'qualityLevel': 0.3,
             'minDistance': 7,
             'blockSize': 7}

# Lucas-Kanade optical flow
config_lk = {'winSize': (15, 15),
             'maxLevel': 2,
             'criteria': (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)}

# Create some random colors
color = np.random.randint(0, 255, (100, 3))


In [5]:
# Create some random colors
color = np.random.randint(0, 255, (100, 3))

# Take first frame and find keypoints

# If the VideoCapture failed to open the provided path, try the local 'data' folder
if not video1.isOpened():
    alt_path = os.path.join('source', 'birds_in_sky.mp4')
    if os.path.exists(alt_path):
        video1 = cv2.VideoCapture(alt_path)

ret, source = video1.read()

assert ret

In [6]:
# Початкова ініціалізація
src_gray = cv2.cvtColor(source, cv2.COLOR_BGR2GRAY)
p_src = cv2.goodFeaturesToTrack(src_gray, mask=None, **config_st)

# Create a mask image for drawing purposes
mask = np.zeros_like(source)


ret, source = video1.read()


frame_idx = 0
gif_frames = [] 

In [7]:


while True:
    ret, target = video1.read()
    if not ret:
        print('End of video.')
        break

    frame_idx += 1
    if frame_idx % 10 != 0:
        continue
    
    dst_gray = cv2.cvtColor(target, cv2.COLOR_BGR2GRAY)
    p_dst, status, err = cv2.calcOpticalFlowPyrLK(src_gray, dst_gray, p_src, None, **config_lk)
    
    # Вибираємо тільки вдало відстежені точки
    if p_dst is not None:
        good_new = p_dst[status==1]
        good_old = p_src[status==1]
    

    # Якщо ми втратили всі точки, шукаємо нові і переходимо до наступної ітерації
    if len(good_new) < 5: # Наприклад, якщо залишилось менше 5 точок
        # Оновлюємо кадр для наступної ітерації
        src_gray = dst_gray.copy()
        # Шукаємо нові точки на поточному кадрі
        p_src = cv2.goodFeaturesToTrack(src_gray, mask=None, **config_st)
        continue # Пропускаємо малювання на цьому кадрі
    # ----------------------------------------------------

    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)    
        target = cv2.circle(target, (int(a), int(b)), 5, color[i].tolist(), -1)

    result = cv2.add(target, mask)
    gif_frames.append(result)
    
    # Оновлюємо попередній кадр та точки
    src_gray = dst_gray.copy()
    p_src = good_new.reshape(-1, 1, 2)

#Створення GIF та звільнення відео ПІСЛЯ циклу ---
if gif_frames:
    gif_filename = 'optical_flow_tracking.gif'
    #gif_filename = 'hawk_tracking.gif'
    output_path = os.path.join(output_folder, gif_filename)
    # --- Попередня перевірка існування теки ---
    if not os.path.exists(output_folder):
        print(f"Creating directory: {output_folder}")
        os.makedirs(output_folder)
    else:
        print(f"Directory '{output_folder}' already exists.")

    print(f"Creating GIF with {len(gif_frames)} frames...")
    create_gif(gif_frames, output_path, fps=5)

video1.release()
# -----------------------------------------------------------------

End of video.
Directory 'result' already exists.
Creating GIF with 34 frames...
Successfully saved GIF to result\optical_flow_tracking.gif


In [8]:
# ===================================================================
# === ЧАСТИНА 2: ВІДСТЕЖЕННЯ ЗА ШАБЛОНОМ (TEMPLATE MATCHING) ===
# ===================================================================
# Заново відкриваємо відеофайл, щоб почати з першого кадру
video_template = cv2.VideoCapture('source//birds_in_sky.mp4')

# Визначаємо рамку для одного з птахів на першому кадрі
x1, y1 = 930, 270
x2, y2 = 1150, 540
width = x2 - x1
height = y2 - y1

# Обмежуємо область пошуку навколо об'єкта
search = 40 # Птахи рухаються швидше, можна збільшити/зменшити

In [9]:
#Створюємо шаблон з першого кадру
ret, first_frame = video_template.read()
if ret:
    template = first_frame[y1:y2, x1:x2]
    #cv2.imshow("Template", template) # Можна розкоментувати, щоб побачити шаблон
    #cv2.waitKey(0)
else:
    print("Failed to read the first frame for template matching.")
    exit()

In [10]:
# Готуємося до циклу відстеження
gif_frames_template = []
# Обмежимо кількість кадрів для відстеження, щоб GIF не був занадто великим
num_frames_to_track = 60 

In [11]:
for i in range(num_frames_to_track):
    ret, frame = video_template.read()
    if not ret:
        print("End of video for template matching.")
        break

    # Визначаємо межі вікна пошуку, щоб не вийти за межі кадру
    h, w, _ = frame.shape
    search_y1 = max(0, y1 - search)
    search_y2 = min(h, y2 + search)
    search_x1 = max(0, x1 - search)
    search_x2 = min(w, x2 + search)
    
    search_window = frame[search_y1:search_y2, search_x1:search_x2]
    
    # Виконуємо зіставлення з шаблоном
    # cv2.TM_CCOEFF_NORMED - один з найнадійніших методів
    res = cv2.matchTemplate(search_window, template, cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    
    # Найкраща позиція знаходиться в max_loc для цього методу
    track_x1, track_y1 = max_loc
    
    # Оновлюємо координати рамки
    x1 = search_x1 + track_x1
    y1 = search_y1 + track_y1
    
    # Малюємо рамку на кадрі
    cv2.rectangle(frame, (x1, y1), (x1 + width, y1 + height), (0, 255, 0), 2)
    gif_frames_template.append(frame)

In [12]:
# Зберігаємо результат у новий GIF-файл
if gif_frames_template:
    gif_filename_template = 'template_tracking.gif'
    output_path_template = os.path.join(output_folder, gif_filename_template)
    
    print(f"Creating template matching GIF with {len(gif_frames_template)} frames...")
    # Використовуємо ту саму функцію create_gif
    create_gif(gif_frames_template, output_path_template, fps=5)

Creating template matching GIF with 60 frames...
Successfully saved GIF to result\template_tracking.gif


In [13]:
# Звільняємо другий відеопотік
video_template.release()
print("Template matching tracking finished.")

Template matching tracking finished.


In [ ]:
# === ЧАСТИНА 3: ВІДСТЕЖЕННЯ ЗА ДОПОМОГОЮ KCF ===
print("\nStarting Part 3: KCF Tracking...")

# 1. Ініціалізація
video_kcf = cv2.VideoCapture('source//birds_in_sky.mp4')
output_folder = 'result'
os.makedirs(output_folder, exist_ok=True)

# 2. Створюємо об'єкт трекера KCF
tracker = cv2.TrackerKCF_create()

# 3. Зчитуємо перший кадр
ret, first_frame = video_kcf.read()
if not ret:
    print("Failed to read video")
    exit()

# 4. Дозволяємо користувачу вибрати об'єкт для відстеження
print("Please select a bounding box for the bird and press ENTER or SPACE.")
bbox = cv2.selectROI("Select Bird to Track", first_frame, False)
cv2.destroyWindow("Select Bird to Track")

# 5. Ініціалізуємо трекер першим кадром та вибраною рамкою
tracker.init(first_frame, bbox)

# 6. Готуємося до циклу
gif_frames_kcf = []

while True:
    ret, frame = video_kcf.read()
    if not ret:
        print("End of video for KCF tracking.")
        break

    # 7. Оновлюємо трекер
    ok, bbox = tracker.update(frame)

    # 8. Малюємо рамку
    if ok:
        # Відстеження успішне
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, p1, p2, (0, 255, 0), 6, 1)
        cv2.putText(frame, "Tracking", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
    else:
        # Відстеження не вдалося
        cv2.putText(frame, "Tracking failure detected", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)

    # Додаємо кадр для GIF
    gif_frames_kcf.append(frame)
    
    # Показуємо результат в реальному часі (опційно)
    cv2.imshow("KCF Tracker", frame)

    # Вихід по натисканню 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 9. Зберігаємо результат у GIF
if gif_frames_kcf:
    gif_filename_kcf = 'kcf_tracking.gif'
    output_path_kcf = os.path.join(output_folder, gif_filename_kcf)
    
    print(f"Creating KCF tracking GIF with {len(gif_frames_kcf)} frames...")
    create_gif(gif_frames_kcf, output_path_kcf, fps=10) # Збільшимо fps для плавності

# 10. Звільняємо ресурси
video_kcf.release()
cv2.destroyAllWindows()
print("KCF tracking finished.")


Starting Part 3: KCF Tracking...
Please select a bounding box for the bird and press ENTER or SPACE.
End of video for KCF tracking.
Creating KCF tracking GIF with 359 frames...
Successfully saved GIF to result\kcf_tracking.gif
KCF tracking finished.
